## EK91 QKD Try

In [1]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
backend = Aer.get_backend('qasm_simulator')

Measure and execute

algo written in quil

In [32]:
def generate_pair():
    
    cct = QuantumCircuit(2,2)
    cct.h(0)
    cct.cx(0,1)
    
    return cct

def measure_A(cct, q):
    basis = np.random.choice([0, 1, 2])

    if basis == 1:
        cct.ry(-np.pi/4, q)

    elif basis == 2:
        cct.ry(-np.pi/2, q)
        
    cct.measure(q,q)
    return cct, basis

def measure_B(cct, q):
    basis = np.random.choice([1, 2, 3])
    if basis == 1:
        cct.ry(-np.pi/4,q)
        
    elif basis == 2:
        cct.ry(-np.pi/2, q)

    elif basis == 3:
        cct.ry(-3*np.pi/4, q)
        
    cct.measure(q,q)
    return cct, basis

def measure_E(cct, pair):
    cct, _ = measure_B(cct, pair[1])
#     pair[1] = QubitPlaceholder()
    return cct

def get_sequences(key_len, eavesdrop):
    alice_bases = []
    bob_bases = []
    alice_received = []
    bob_received = []
    for _ in range(round(9 * key_len / 2)):

        cct = generate_pair()
        cct_a = cct.copy()
        cct_b = cct.copy()
        pair = [0,1] #0: Alice, 1: Bob

        if eavesdrop:
            cct = measure_E(cct, pair)
        cct_a, basis_a = measure_A(cct_a, pair[0])
        cct_b, basis_b = measure_B(cct_b, pair[1])

        cct.compose(cct_a, inplace=True)
        cct.compose(cct_b, inplace=True)
        
        #run on Aer simulator   
        result = execute(cct, backend, shots=1).result()
        bits = result.get_counts(cct)
        bits = next(iter(bits))
        
        alice_bases.append(basis_a)
        bob_bases.append(basis_b)
        
        alice_received.append(int(bits[-1]))
        bob_received.append(int(bits[-2]))
    return np.asarray(alice_bases), np.asarray(bob_bases), np.asarray(alice_received), np.asarray(bob_received)

def correlation(obs1, obs2, bases1, bases2, i, j):
    match1 = np.nonzero(bases1 == i)
    match2 = np.nonzero(bases2 == j)
    matches = np.intersect1d(match1, match2)
    e1 = np.sum(np.logical_not(np.logical_xor(obs1[matches],  obs2[matches])))
    e2 = np.sum(np.logical_xor(obs1[matches],  obs2[matches]))
    return (e1 - e2) / (e1 + e2) if matches.shape[0] > 0 else 0

def verify(obs1, obs2, bases1, bases2):
    S = correlation(obs1, obs2, bases1, bases2, 0, 1)
    S -= correlation(obs1, obs2, bases1, bases2, 0, 3)
    S += correlation(obs1, obs2, bases1, bases2, 2, 1)
    S += correlation(obs1, obs2, bases1, bases2, 2, 3)
    #print(S)
    return S > 2

def generate_key(key_len, eavesdrop=False):
    alice_bases, bob_bases, alice_received, bob_received = get_sequences(key_len, eavesdrop)
    matching = (alice_bases == bob_bases)
    alice_key = alice_received[matching]
    bob_key = bob_received[matching]
    secure = verify(alice_received, bob_received, alice_bases, bob_bases)
    #assert (alice_key == bob_key).all()
    return alice_key, bob_key, secure

trial_sizes = list(range(1, 21))
trial_sizes.extend([25, 30]) #40, 50])#, 75, 100, 150, 200])
false_poss = []
false_negs = []
for num in trial_sizes:
    print("Trial {}".format(num))
    false_neg = 0
    false_pos = 0
    for _ in range(100):
        _, _, secure = generate_key(num, False)
        if not secure:
            false_pos += 1
        _, _, secure = generate_key(num, True)
        if secure:
            false_neg += 1
    false_poss.append(false_pos / 200)
    false_negs.append(false_neg / 200)

plt.plot(trial_sizes, false_poss, label="False Positive Rate")
plt.plot(trial_sizes, false_negs, label="False Negative Rate")
plt.legend()
plt.title("QKD Performance")
plt.xlabel("Number of Qubit Pairs Exchanged")
plt.ylabel("Probability")
plt.show()

Trial 1
Trial 2


KeyboardInterrupt: 